In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
import yfinance as yf
from pickle import load
from datetime import datetime, timedelta
from numpy import array as np_array, concatenate, zeros


MODEL_PATH = "/Users/nik/PycharmProjects/sibyl/backend/src/oracle/models/oracle_v0.1.keras"
SCALER_PATH = "/Users/nik/PycharmProjects/sibyl/backend/src/oracle/preprocessing/minmax_scaler-7f.pkl"


class Oracle:
    def __init__(self):
        self.lookback = 60  # days
        self.forecast_horizon = 7  # days
        self.model = load_model(MODEL_PATH)
        self.scaler = load(open(SCALER_PATH, 'rb'))
        self.dataset = pd.DataFrame()
        self.model_input = None
        self.current_btc_price = None
        
        
    def fetch_dataset(self):

        start_date = (datetime.today() - timedelta(days=self.lookback)).strftime('%Y-%m-%d')
        end_date = datetime.today().strftime('%Y-%m-%d')

        bitcoin = yf.download('BTC-USD', start=start_date, end=end_date)
        self.dataset['bitcoin_price'] = bitcoin['Adj Close']
        self.current_btc_price = self.dataset['bitcoin_price'].values
        sp500 = yf.download('^GSPC', start=start_date, end=end_date)
        self.dataset['sp500_price'] = sp500['Adj Close']
        gold = yf.download('GC=F', start=start_date, end=end_date)
        self.dataset['gold_price'] = gold['Adj Close']
        usd_index = yf.download('DX-Y.NYB', start=start_date, end=end_date)
        self.dataset['usd_index'] = usd_index['Adj Close']
        vol_index = yf.download('^VIX', start=start_date, end=end_date)
        self.dataset['volatility_index'] = vol_index['Adj Close']
        irx = yf.download('^IRX', start=start_date, end=end_date)
        self.dataset['interest_rate'] = irx['Adj Close']
        oil = yf.download('CL=F', start=start_date, end=end_date)
        self.dataset['oil_price'] = oil['Adj Close']

        self.dataset.ffill(inplace=True)
        self.dataset.dropna(inplace=True)

    def dataset_preprocessing(self):
        self.dataset = self.scaler.transform(self.dataset[['bitcoin_price', 'sp500_price', 'gold_price', 'usd_index', 'oil_price',
                                               'volatility_index', 'interest_rate']])
        
    def create_model_input(self):            
        self.model_input = self.dataset.reshape(1, self.lookback, self.dataset.shape[1])
    
    def generate_model_prediction(self):
        y_pred = self.model.predict(self.model_input)
        dummy_2d_array = zeros((self.forecast_horizon, 7))
        dummy_2d_array[:, 0] = y_pred[0]
        rescaled_2d_array = self.scaler.inverse_transform(dummy_2d_array)
        return rescaled_2d_array[:, 0]
    
    def generate_btc_prediction(self):
        self.fetch_dataset()
        self.dataset_preprocessing()
        self.create_model_input()
        y_pred = self.generate_model_prediction()
        return self.current_btc_price, y_pred

In [ ]:
client = Oracle()
a = client.generate_btc_prediction()
print(a)

In [ ]:
b = list(a[0]) + list(a[1])
print(b)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(a[1])

In [ ]:

# Generate the list of dictionaries with date and price
result = []
time_diff = len(b)
for i, price in enumerate(b):
    result.append({"date": (datetime.today() - timedelta(days=time_diff)).strftime('%Y-%m-%d'), "price": price})
    time_diff -= 1

# Generate the list of date and price data
import plotly.graph_objects as go

# Convert to a DataFrame for Plotly
df = pd.DataFrame(result)

fig = go.Figure()

# Plot actual data
fig.add_trace(go.Scatter(x=df[0:60]['date'], y=df[0:60]['price'],
                         mode='lines', name='Actual',
                         line=dict(color='blue', width=2)))

# Plot predicted data
fig.add_trace(go.Scatter(x=df[59:-1]['date'], y=df[59:-1]['price'],
                         mode='lines+markers', name='Prediction',
                         line=dict(color='rgba(255, 165, 0, 0.8)', width=2, shape='spline'),  # Smooth curve with transparency
                         marker=dict(size=6, color='orange', symbol='circle', line=dict(width=2, color='black')),
                         fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))  # Gradient-like effect


# Update layout
fig.update_layout(
    title='Price and Oracle Prediction',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_tickangle=-45,
    hovermode='closest'  # Smooth hover interaction
)

# Show the plot
fig.show()

In [ ]:
result = []
time_diff = len(b)-7
for i, price in enumerate(b):
    result.append({"date": (datetime.today() - timedelta(days=time_diff)).strftime('%Y-%m-%d'), "price": price})
    time_diff -= 1
result